In [37]:
# import packages used in this notebook
import os
from zipfile import ZipFile 
import json

In [38]:
import pyspark
from pyspark.sql import SparkSession
sc = SparkSession.builder.getOrCreate()

In [39]:
tweets = [] # list of tweets

In [40]:
def getbz2Tweets(files, subject):
    import bz2
    import re
    subj = subject.lower()
    with bz2.open(files, "rt", encoding="utf-8") as f:
    # error encountered when loading the entire file as json object
        for line in f:
            try:
                data = json.loads(line)
                # filter the data for my subject
                fltrData = re.search(r'\b{}\b'.format(subj), line.lower())

                if fltrData:
                    try:
                        tweet = {'id':[data["id"]],
                              'text': data["text"]}
                
                        # user = data["user"]
                        # tweet['userName'] = user["name"]
                        # tweet['userLocation'] = user["location"]
                        # tweet['userFollowers'] = user["followers_count"]
                        # if 'retweeted_status' in data:
                        #     retweet = data["retweeted_status"]
                        #     tweet['retweetUserName'] = retweet["user"]
                        #     tweet['retweetCount'] = retweet["retweet_count"]
                        tweets.append(tweet)
                    except KeyError as e:
                        #print("KeyError encountered", {e}) # error is encountered mainly due to deleted tweets 
                        #print(line)
                        continue
            except json.decoder.JSONDecodeError as e:
                print("Error encountered at line: ", {e})
                continue
    return tweets
   

In [41]:
def getgzTweets(files, subject):
    import gzip
    import re
    subj = subject.lower()
    with gzip.open(files, "rt", encoding="utf-8") as f:
    # error encountered when loading the entire file as json object
        for line in f:
            try:
                data = json.loads(line)
                # filter the data for my subject
                fltrData = re.search(r'\b{}\b'.format(subj), line.lower())

                if fltrData:
                    print(line)
                    try:
                        tweet = {'id':[data["id"]],
                              'text': data["text"]}
                
                        # user = data["user"]
                        # tweet['userName'] = user["name"]
                        # tweet['userLocation'] = user["location"]
                        # tweet['userFollowers'] = user["followers_count"]
                        # if 'retweeted_status' in data:
                        #     retweet = data["retweeted_status"]
                        #     tweet['retweetUserName'] = retweet["user"]
                        #     tweet['retweetCount'] = retweet["retweet_count"]
                        tweets.append(tweet)
                    except KeyError as e:
                        #print("KeyError encountered", {e}) # error is encountered mainly due to deleted tweets 
                        #print(line)
                        continue
            except json.decoder.JSONDecodeError as e:
                print("Error encountered at line: ", {e})
                continue
    return tweets
   

In [31]:
# tweetDrv = "/home/sba22230/shared-drives/E:/TwitterStream"
tweetDrv = "./SampleTweets/"

tweetWrappers = os.listdir(tweetDrv)

display(tweetWrappers)


['20221101000000.json',
 'twitter-stream-2021-01-01.zip',
 'onetweet.txt',
 '29.json',
 '20221101235900.json']

In [32]:
# Deal with the zip and tar wrapper 
for wrap in tweetWrappers:
    file = tweetDrv + wrap
    if file.endswith('.zip'):
        print("Zip file found")
        with ZipFile(file, 'r') as zip:
            tweetFiles = zip.printdir()
            for filbz in tweetFiles:
                d
                filbz = tweetDrv + file + filbz
                if filbz.endswith('.bz2'):
                    tweets.append(getbz2Tweets(filbz, 'euro'))
                elif filbz.endswith('.gz'):
                    tweets.append(getgzTweets(filbz, 'euro'))
    elif file.endswith('.tar'):
        print("Tar file found")
    else:
        print("unknown file" + os.path.splitext(file)[-1])

unknown file.json
Zip file found
File Name                                             Modified             Size
2021/01/01/                                    2021-01-02 06:00:06            0
2021/01/01/19/                                 2021-01-03 03:30:02            0
2021/01/01/19/31.json.bz2                      2021-01-02 08:39:46      1391786
2021/01/01/19/43.json.bz2                      2021-01-02 08:39:48      1372842
2021/01/01/19/15.json.bz2                      2021-01-02 08:39:42      1489551
2021/01/01/19/39.json.bz2                      2021-01-02 08:39:48      1385437
2021/01/01/19/26.json.bz2                      2021-01-02 08:39:46      1386606
2021/01/01/19/28.json.bz2                      2021-01-02 08:39:46      1416889
2021/01/01/19/53.json.bz2                      2021-01-02 08:39:50      1403190
2021/01/01/19/33.json.bz2                      2021-01-02 08:39:46      1351820
2021/01/01/19/48.json.bz2                      2021-01-02 08:39:50      1380512
2021/01

TypeError: 'NoneType' object is not iterable

In [36]:
zip.infolist()

[<ZipInfo filename='2021/01/01/' filemode='drwx------' external_attr=0x10>,
 <ZipInfo filename='2021/01/01/19/' filemode='drwx------' external_attr=0x10>,
 <ZipInfo filename='2021/01/01/19/31.json.bz2' compress_type=deflate filemode='-rw-------' file_size=1391786 compress_size=1391396>,
 <ZipInfo filename='2021/01/01/19/43.json.bz2' compress_type=deflate filemode='-rw-------' file_size=1372842 compress_size=1372544>,
 <ZipInfo filename='2021/01/01/19/15.json.bz2' compress_type=deflate filemode='-rw-------' file_size=1489551 compress_size=1489001>,
 <ZipInfo filename='2021/01/01/19/39.json.bz2' compress_type=deflate filemode='-rw-------' file_size=1385437 compress_size=1384910>,
 <ZipInfo filename='2021/01/01/19/26.json.bz2' compress_type=deflate filemode='-rw-------' file_size=1386606 compress_size=1386045>,
 <ZipInfo filename='2021/01/01/19/28.json.bz2' compress_type=deflate filemode='-rw-------' file_size=1416889 compress_size=1416315>,
 <ZipInfo filename='2021/01/01/19/53.json.bz2' 

In [33]:
display(tweets[0]['id'])

IndexError: list index out of range

In [ ]:
idList = [tweet['id'] for tweet in tweets]
if [1410485572347387910] in idList:
    print("True")

In [ ]:
display(tweets)

In [ ]:
type(tweets[0])